In [1]:
from find_source import summary
from catalogs import low_level_table, high_level_table, light_curve
from obs_html import full_html_and_txt
import glob
from astropy.io import fits

In [2]:
#full_html_and_txt('../data/250611_03:56:34')

In [3]:
#for file in glob.glob('../data/multi_track/*.fits'):
#    try:
#        summary(fits_file=file, short_dict=False, plot=True, save_path='../test')
#    except Exception as e:
#        print(f'Error for {file}: {e}')

In [4]:
for folder in glob.glob('/mnt/COMPASS9/sma/quality/*'):
    try:
        low_level_table(folder)
    except Exception as e:
        print(f'{e} for {folder}')

/py3opt/anaconda3/lib/python3.11/site-packages/astropy/units/quantity.py:1350: ComplexWarning: Casting complex values to real discards the imaginary part
  return float(self.to_value(dimensionless_unscaled))


Error for /mnt/COMPASS9/sma/quality/11306/1743-038_img_RX3.fits: float() argument must be a string or a real number, not 'complex'
Error for /mnt/COMPASS9/sma/quality/13513/1658+347_img_RX0.fits: index 0 is out of bounds for axis 0 with size 0
Error for /mnt/COMPASS9/sma/quality/13316/1127-189_img_RX3.fits: index 0 is out of bounds for axis 0 with size 0
Error for /mnt/COMPASS9/sma/quality/13316/1130-148_img_RX3.fits: index 0 is out of bounds for axis 0 with size 0
Error for /mnt/COMPASS9/sma/quality/13316/3c279_img_RX3.fits: index 0 is out of bounds for axis 0 with size 0


/py3opt/anaconda3/lib/python3.11/site-packages/astropy/units/quantity.py:1350: ComplexWarning: Casting complex values to real discards the imaginary part
  return float(self.to_value(dimensionless_unscaled))


Error for /mnt/COMPASS9/sma/quality/10618/1146+399_img_RX0.fits: float() argument must be a string or a real number, not 'complex'
No values after mask applied. Check inclusion and exclusion radii.
Error for /mnt/COMPASS9/sma/quality/10618/1146+399_img_RX3.fits: cannot access local variable 'peak' where it is not associated with a value
Error for /mnt/COMPASS9/sma/quality/10618/1159+292_img_RX0.fits: float division by zero


In [12]:
high_level_table()

In [6]:
#light_curve()

In [7]:
#hdul = fits.open('../data/11306/3c273_img_RX3.fits')

In [8]:
#hdul[1].columns[1].name
#hdul[1].columns[1].unit
#hdul[1].data[1][1]

In [9]:
#hdul[1].columns

In [10]:
#float(hdul[1].data[0][2])

In [11]:
#summary('../data/11306/3c273_img_RX3.fits')